# Phase 3: Experiments on Unlabeled Data

## 1. Setup & Configuration

In [33]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
import random
from copy import deepcopy
from collections import defaultdict
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision
from torchvision import transforms

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm.auto import tqdm
import cv2
from PIL import Image
from pathlib import Path

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Imports successful!')

✓ Imports successful!


In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Device: cuda
GPU: NVIDIA L4


In [63]:
# Configuration
NUM_CLASSES = 20

CLASS_NAMES = {
    0: 'Groove_billed_Ani', 1: 'Red_winged_Blackbird', 2: 'Rusty_Blackbird',
    3: 'Gray_Catbird', 4: 'Brandt_Cormorant', 5: 'Eastern_Towhee',
    6: 'Indigo_Bunting', 7: 'Brewer_Blackbird', 8: 'Painted_Bunting',
    9: 'Bobolink', 10: 'Lazuli_Bunting', 11: 'Yellow_headed_Blackbird',
    12: 'American_Crow', 13: 'Fish_Crow', 14: 'Brown_Creeper',
    15: 'Yellow_billed_Cuckoo', 16: 'Yellow_breasted_Chat',
    17: 'Black_billed_Cuckoo', 18: 'Gray_crowned_Rosy_Finch', 19: 'Bronzed_Cowbird'
}

CONFIG = {
    # Competition data
    'data_dir': '/content/drive/MyDrive/data',
    'train_csv': '/content/drive/MyDrive/data/train_metadata.csv',
    'val_csv': '/content/drive/MyDrive/data/val_metadata.csv',
    'combined_csv': '/content/drive/MyDrive/data/combined_metadata.csv',
    'train_images': '/content/drive/MyDrive/data/train_images',
    'val_images': '/content/drive/MyDrive/data/val_images',
    'combined_images': '/content/drive/MyDrive/data/combined_images',
    'test_images': '/content/drive/MyDrive/data/test_images/mistery_cat',
    'sample_submission': '/content/drive/MyDrive/data/sample_submission.csv',

    # External unlabeled data (if available)
    'external_unlabeled_dir': '/content/drive/MyDrive/data/external_birds_unlabeled',
    'use_external_data': True,

    # Filtered pseudo-labeled data directory
    'filtered_external_dir': '/content/drive/MyDrive/data/filtered_external_birds',

    # Phase 2 checkpoint (REQUIRED for pseudo-labeling)
    'phase2_checkpoint_dir': '/content/drive/MyDrive/ml_results/phase2b_checkpoints',
    'phase2_best_model': '/content/drive/MyDrive/ml_results/phase2b_checkpoints/swin_base_224_best.pth',

    # Training params
    'img_size': 224,  #
    'batch_size': 20,
    'epochs': 50,
    'lr': 3e-5,  # Fine-tuning rate
    'weight_decay': 1e-4,
    'num_workers': 4,

    # Pseudo-labeling params
    'pseudo_label_confidence': 0.9,  # High confidence threshold
    'pseudo_label_max_images': 5000,  # Max pseudo-labeled images
    'filter_batch_size': 32,

    # Optional: Train multiple architectures for diversity
    'train_multiple_architectures': True,  # Set True for architecture ensemble
    'architectures': ['resnet50', 'vit_small_patch16_224', 'convnext_tiny'],

    # Output
    'checkpoint_dir': '/content/drive/MyDrive/ml_results/phase3_checkpoints',
    'results_dir': '/content/drive/MyDrive/ml_results/phase3_results',
}

os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)
os.makedirs(CONFIG['results_dir'], exist_ok=True)
os.makedirs(CONFIG['filtered_external_dir'], exist_ok=True)

print('✓ Phase 3 Streamlined Configuration')
print(f"Image size: {CONFIG['img_size']}")
print(f"Epochs: {CONFIG['epochs']}")
print(f"Pseudo-label threshold: {CONFIG['pseudo_label_confidence']}")
print(f"Multiple architectures: {CONFIG['train_multiple_architectures']}")

✓ Phase 3 Streamlined Configuration
Image size: 224
Epochs: 50
Pseudo-label threshold: 0.9
Multiple architectures: True


## 2. STAGE 1: Smart Pseudo-Labeling Filter

**The KEY technique:** Filter external data to only your 20 target species

In [55]:
def load_phase2_model():
    """
    Load Phase 2 best model for filtering
    """
    model = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=NUM_CLASSES)

    checkpoint_path = CONFIG['phase2_best_model']
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Phase 2 checkpoint not found: {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path, map_location=device)
    state_dict = checkpoint.get('model_state_dict', checkpoint)
    model.load_state_dict(state_dict)
    model = model.to(device)
    model.eval()

    print(f"✓ Phase 2 model loaded from {checkpoint_path}")
    return model

# Load Phase 2 model
if CONFIG['use_external_data']:
    phase2_model = load_phase2_model()
else:
    print("Skipping external data filtering (use_external_data=False)")

✓ Phase 2 model loaded from /content/drive/MyDrive/ml_results/phase2b_checkpoints/swin_base_224_best.pth


In [38]:
def filter_external_data_with_pseudolabels(model, confidence_threshold=0.95, max_images=5000):
    """
    Filter external unlabeled bird images to only keep those predicted as our 20 classes

    This is the solution to the dataset mismatch problem:
    - NABirds has 400+ species → We keep only our 20
    - iNaturalist has 1000+ species → We keep only our 20

    Args:
        model: Trained Phase 2 model
        confidence_threshold: Minimum confidence (0.95 = very strict)
        max_images: Maximum images to keep per class

    Returns:
        DataFrame with filtered images and pseudo-labels
    """
    external_dir = CONFIG['external_unlabeled_dir']

    if not os.path.exists(external_dir):
        print(f"⚠️ External data directory not found: {external_dir}")
        print("Continuing without external data...")
        return pd.DataFrame()

    # Find all images
    image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
    all_images = []

    for root, dirs, files in os.walk(external_dir):
        for file in files:
            if Path(file).suffix in image_extensions:
                all_images.append(os.path.join(root, file))

    print(f"Found {len(all_images)} images in external directory")

    if len(all_images) == 0:
        print("No images found. Continuing without external data...")
        return pd.DataFrame()

    # Inference transform
    transform = A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

    # Storage
    filtered_data = []

    model.eval()
    print(f"\nFiltering with confidence > {confidence_threshold}...")

    with torch.no_grad():
        for img_path in tqdm(all_images, desc="Filtering external images"):
            try:
                image = cv2.imread(img_path)
                if image is None:
                    continue
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                augmented = transform(image=image)['image']
                augmented = augmented.unsqueeze(0).to(device)

                output = model(augmented)
                probs = F.softmax(output, dim=1)
                confidence, pred_class = torch.max(probs, dim=1)

                confidence = confidence.item()
                pred_class = pred_class.item()

                # Keep only high-confidence predictions
                if confidence >= confidence_threshold:
                    filtered_data.append({
                        'path': img_path,
                        'pseudo_label': pred_class,
                        'confidence': confidence
                    })

            except Exception as e:
                continue

    df = pd.DataFrame(filtered_data)

    if len(df) == 0:
        print("⚠️ No high-confidence images found!")
        print("Try lowering pseudo_label_confidence to 0.90 or 0.85")
        return df

    # Balance classes - keep max N images per class
    balanced_data = []
    for class_idx in range(NUM_CLASSES):
        class_df = df[df['pseudo_label'] == class_idx].sort_values('confidence', ascending=False)
        max_per_class = max_images // NUM_CLASSES
        balanced_data.append(class_df.head(max_per_class))

    df = pd.concat(balanced_data, ignore_index=True)

    print(f"\n✓ Filtered {len(df)} high-confidence images (balanced)")
    print(f"Confidence range: {df['confidence'].min():.4f} - {df['confidence'].max():.4f}")
    print(f"\nPseudo-label distribution:")
    print(df['pseudo_label'].value_counts().sort_index())

    # Save
    df.to_csv(os.path.join(CONFIG['filtered_external_dir'], 'pseudo_labels.csv'), index=False)

    return df

# Run filtering
if CONFIG['use_external_data']:
    pseudo_labeled_df = filter_external_data_with_pseudolabels(
        phase2_model,
        confidence_threshold=CONFIG['pseudo_label_confidence'],
        max_images=CONFIG['pseudo_label_max_images']
    )
else:
    pseudo_labeled_df = pd.DataFrame()
    print("Skipping pseudo-labeling (use_external_data=False)")

Found 2403 images in external directory

Filtering with confidence > 0.9...


Filtering external images:   0%|          | 0/2403 [00:00<?, ?it/s]


✓ Filtered 798 high-confidence images (balanced)
Confidence range: 0.9001 - 0.9904

Pseudo-label distribution:
pseudo_label
0       2
1      83
2      23
3      37
4      30
5      45
6      55
7      48
8     103
9      63
10      9
11     53
12      1
13     10
14     90
15     31
16     63
17     25
18     16
19     11
Name: count, dtype: int64


## 3. Data Loading - Competition + Filtered Data

In [56]:
# Load competition data
train_df = pd.read_csv(CONFIG['train_csv'])
val_df = pd.read_csv(CONFIG['val_csv'])
combined_df = pd.read_csv(CONFIG['combined_csv'])

print(f"Competition training: {len(train_df)}")
print(f"Competition validation: {len(val_df)}")
print(f"Combined samples: {len(combined_df)}")

if len(pseudo_labeled_df) > 0:
    print(f"Filtered pseudo-labeled: {len(pseudo_labeled_df)}")
    print(f"Total training: {len(train_df) + len(pseudo_labeled_df)}")
    boost = (len(pseudo_labeled_df) / len(train_df)) * 100
    print(f"Data boost: +{boost:.1f}%")
else:
    print("No pseudo-labeled data")

Competition training: 1082
Competition validation: 103
Combined samples: 1185
Filtered pseudo-labeled: 798
Total training: 1880
Data boost: +73.8%


In [57]:
# STRONG augmentations for Phase 3
train_transform = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),

    # Geometric
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.6),
    A.Perspective(scale=(0.05, 0.1), p=0.3),

    # Blur/Noise
    A.OneOf([
        A.GaussNoise(var_limit=(10, 50)),
        A.GaussianBlur(blur_limit=(3, 7)),
        A.MotionBlur(blur_limit=7),
    ], p=0.3),

    # Distortion
    A.OneOf([
        A.OpticalDistortion(distort_limit=0.3),
        A.GridDistortion(num_steps=5, distort_limit=0.3),
        A.ElasticTransform(alpha=1, sigma=50),
    ], p=0.3),

    # Color
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),

    # Cutout
    A.CoarseDropout(max_holes=8, max_height=40, max_width=40, p=0.3),

    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

print('✓ Strong augmentation pipelines ready')

✓ Strong augmentation pipelines ready


In [58]:
class BirdDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_pseudo=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.is_pseudo = is_pseudo

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        if self.is_pseudo:
            img_path = row['path']
            label = row['pseudo_label']
        else:
            img_path = os.path.join(self.img_dir, row['path'])
            label = row['class_idx']

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label

# Create datasets
train_dataset = BirdDataset(train_df, CONFIG['combined_images'], train_transform)
val_dataset = BirdDataset(val_df, CONFIG['val_images'], val_transform)

# Add pseudo-labeled data if available
if len(pseudo_labeled_df) > 0:
    pseudo_dataset = BirdDataset(pseudo_labeled_df, None, train_transform, is_pseudo=True)
    train_dataset = ConcatDataset([train_dataset, pseudo_dataset])
    print(f"✓ Combined dataset: {len(train_dataset)} images")

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'],
                         shuffle=True, num_workers=CONFIG['num_workers'], pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'],
                       shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

✓ Combined dataset: 1880 images
Training batches: 94
Validation batches: 6


## 4. STAGE 2: Advanced Training with MixUp + CutMix

In [59]:
def mixup_data(x, y, alpha=0.4):
    """MixUp augmentation"""
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def cutmix_data(x, y, alpha=1.0):
    """CutMix augmentation"""
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    _, _, H, W = x.size()
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    x[:, :, bby1:bby2, bbx1:bbx2] = x[index, :, bby1:bby2, bbx1:bbx2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (W * H))

    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

def mixed_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

print('✓ MixUp + CutMix ready')

✓ MixUp + CutMix ready


In [60]:
def train_single_model(model_name='convnext_tiny'):
    """
    Train ONE excellent model with all techniques
    """
    print(f"\n{'='*60}")
    print(f"Training: {model_name}")
    print(f"Image size: {CONFIG['img_size']}")
    print(f"Epochs: {CONFIG['epochs']}")
    print(f"{'='*60}\n")

    # Create model
    model = timm.create_model(model_name, pretrained=True, num_classes=NUM_CLASSES)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['lr'],
                                 weight_decay=CONFIG['weight_decay'])

    # Cosine annealing with warmup
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=CONFIG['epochs'], eta_min=1e-7
    )

    best_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(CONFIG['epochs']):
        # Training
        model.train()
        train_loss = 0.0

        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{CONFIG["epochs"]}')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Random choice: MixUp, CutMix, or neither
            r = np.random.random()
            if r < 0.3:  # 30% MixUp
                images, labels_a, labels_b, lam = mixup_data(images, labels)
                outputs = model(images)
                loss = mixed_criterion(criterion, outputs, labels_a, labels_b, lam)
            elif r < 0.6:  # 30% CutMix
                images, labels_a, labels_b, lam = cutmix_data(images, labels)
                outputs = model(images)
                loss = mixed_criterion(criterion, outputs, labels_a, labels_b, lam)
            else:  # 40% normal
                outputs = model(images)
                loss = criterion(outputs, labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = accuracy_score(all_labels, all_preds)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        scheduler.step()

        print(f"Epoch {epoch+1}/{CONFIG['epochs']}:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Val Acc: {val_acc:.4f}")
        print(f"  LR: {scheduler.get_last_lr()[0]:.2e}")

        # Save best
        if val_acc > best_acc:
            best_acc = val_acc
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }
            save_name = model_name.replace('/', '_')
            torch.save(checkpoint, os.path.join(CONFIG['checkpoint_dir'], f'{save_name}_best.pth'))
            print(f"  ✓ New best: {best_acc:.4f}")

    return model, history, best_acc, model_name

print('✓ Training function ready')

✓ Training function ready


## 5. Train Model(s)

**Option 1:** Train single best architecture  
**Option 2:** Train 2-3 different architectures for real diversity

In [64]:
trained_models = []

if CONFIG['train_multiple_architectures']:
    # Train multiple DIFFERENT architectures (real diversity)
    print("Training multiple architectures for ensemble...")
    for arch in CONFIG['architectures']:
        model, history, best_acc, name = train_single_model(arch)
        trained_models.append({
            'architecture': name,
            'best_acc': best_acc,
            'model': model
        })
else:
    # Train single best model
    print("Training single best model...")
    model, history, best_acc, name = train_single_model('swin_base_patch4_window7_224')
    trained_models.append({
        'architecture': name,
        'best_acc': best_acc,
        'model': model
    })

# Results
results_df = pd.DataFrame(trained_models)
print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(results_df[['architecture', 'best_acc']].to_string(index=False))
print(f"\nBest accuracy: {results_df['best_acc'].max():.4f}")

Training multiple architectures for ensemble...

Training: resnet50
Image size: 224
Epochs: 50



model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Epoch 1/50:   0%|          | 0/94 [00:00<?, ?it/s]

Epoch 1/50:
  Train Loss: 2.9803
  Val Loss: 2.9931
  Val Acc: 0.0291
  LR: 3.00e-05
  ✓ New best: 0.0291


Epoch 2/50:   0%|          | 0/94 [00:00<?, ?it/s]

...

Epoch 49/50:
  Train Loss: 0.9776
  Val Loss: 0.7385
  Val Acc: 0.9126
  LR: 1.30e-07


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c8e5090be20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
Exception ignored in:   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x7c8e5090be20>
    if w.is_alive():Traceback (most recent call last):

  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
        
self._shutdown_workers()   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
  Traceback (most recent call last):
    self._shutdown_workers()    File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
^^^
^   File "/usr/local/lib/python3.12/dist-packages/torch/util

Epoch 50/50:   0%|          | 0/94 [00:00<?, ?it/s]

^^ ^
 ^  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
     ^ assert self._parent_pid == os.getpid(), 'can only test a child process' ^
^^
   File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
 ^     assert self._parent_pid == os.getpid(), 'can only test a child process'^ 
^  ^ ^   Exception ignored in: ^  ^  ^   ^^ ^^^^
^  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
 ^     ^assert self._parent_pid == os.getpid(), 'can only test a child process'<function _MultiProcessingDataLoaderIter.__del__ at 0x7c8e5090be20> ^

^^ Traceback (most recent call last):
^ ^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
     ^^ ^^self._shutdown_workers()^ ^
 ^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
^^ ^     ^^ if w.is_alive():^^ 
^ ^^^^^^ ^^^^ ^^^ ^^ ^^ ^^^ ^^^ ^^^^^^^^^^^^^^^^^^^^^^^^^^^

Epoch 50/50:
  Train Loss: 0.8929
  Val Loss: 0.7379
  Val Acc: 0.9126
  LR: 1.00e-07

TRAINING COMPLETE
         architecture  best_acc
             resnet50  0.854369
vit_small_patch16_224  0.961165
        convnext_tiny  0.941748

Best accuracy: 0.9612


## 6. STAGE 3: Multi-Scale TTA for Final Predictions

In [65]:
def load_best_models():
    """
    Load all trained models
    """
    models = []

    if CONFIG['train_multiple_architectures']:
        for arch in CONFIG['architectures']:
            model = timm.create_model(arch, pretrained=False, num_classes=NUM_CLASSES)
            checkpoint_path = os.path.join(
                CONFIG['checkpoint_dir'],
                f"{arch.replace('/', '_')}_best.pth"
            )
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            model = model.to(device)
            model.eval()
            models.append(model)
    else:
        model = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=NUM_CLASSES)
        checkpoint_path = os.path.join(CONFIG['checkpoint_dir'], 'swin_base_patch4_window7_224_best.pth')
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        model = model.to(device)
        model.eval()
        models.append(model)

    print(f"✓ Loaded {len(models)} model(s)")
    return models

final_models = load_best_models()

✓ Loaded 3 model(s)


In [66]:
def predict_with_multiscale_tta(models, image, scales=[288, 320, 352], n_tta=5):
    """
    Multi-scale + TTA prediction

    Args:
        models: List of models
        image: Input image (H, W, 3) RGB
        scales: Image sizes to try
        n_tta: Number of TTA augmentations
    """
    tta_transforms = [
        A.Compose([A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
        A.Compose([A.HorizontalFlip(p=1.0), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
        A.Compose([A.Rotate(limit=10, p=1.0), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
        A.Compose([A.Rotate(limit=-10, p=1.0), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
        A.Compose([A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=5, p=1.0),
                   A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
    ]

    all_predictions = []

    with torch.no_grad():
        for model in models:
            for scale in scales:
                for tta in tta_transforms[:n_tta]:
                    # Resize and augment
                    resized = cv2.resize(image, (scale, scale))
                    augmented = tta(image=resized)['image']
                    augmented = augmented.unsqueeze(0).to(device)

                    output = model(augmented)
                    probs = F.softmax(output, dim=1)
                    all_predictions.append(probs.cpu().numpy())

    # Average all predictions
    avg_pred = np.mean(all_predictions, axis=0)
    return avg_pred

print('✓ Multi-scale TTA function ready')
print(f"Total predictions per image: {len(final_models)} models × 3 scales × 5 TTA = {len(final_models) * 3 * 5}")

✓ Multi-scale TTA function ready
Total predictions per image: 3 models × 3 scales × 5 TTA = 45


## 7. Generate Final Submission

In [67]:
# Generate submission
test_df = pd.read_csv(CONFIG['sample_submission'])
predictions = []

print("\n" + "="*60)
print("GENERATING FINAL PREDICTIONS")
print(f"Models: {len(final_models)}")
print("Multi-scale TTA: 3 scales × 5 augmentations")
print(f"Total predictions per image: {len(final_models) * 3 * 5}")
print("="*60 + "\n")

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting"):
    img_path = os.path.join(CONFIG['test_images'], row['path'])
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    avg_pred = predict_with_multiscale_tta(
        final_models,
        image,
        scales=[224],
        n_tta=5
    )
    pred_class = np.argmax(avg_pred)
    predictions.append(pred_class)

# Create submission
submission = pd.DataFrame({
    'path': test_df['path'],
    'class_idx': predictions
})

submission_path = os.path.join(
    CONFIG['results_dir'],
    'submission_phase3_streamlined.csv'
)
submission.to_csv(submission_path, index=False)

print(f"\n✓ Submission saved: {submission_path}")
print(f"Total predictions: {len(predictions)}")
print(f"\nPrediction distribution:")
print(pd.Series(predictions).value_counts().sort_index())


GENERATING FINAL PREDICTIONS
Models: 3
Multi-scale TTA: 3 scales × 5 augmentations
Total predictions per image: 45



Predicting:   0%|          | 0/400 [00:00<?, ?it/s]


✓ Submission saved: /content/drive/MyDrive/ml_results/phase3_results/submission_phase3_streamlined.csv
Total predictions: 400

Prediction distribution:
0     21
1     14
2     29
3     19
4     20
5     19
6     15
7     15
8     20
9     28
10    17
11    18
12    13
13    21
14    22
15    34
16    22
17    17
18    19
19    17
Name: count, dtype: int64
